In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from math import log2

# Load the train and test datasets
train_data = pd.read_csv('train.csv', header=None)
test_data = pd.read_csv('test.csv', header=None)

# Assign column names based on the data description
columns = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 
               'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']
train_data.columns = columns
test_data.columns = columns

# Identify numerical columns
numerical_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

# Convert numerical columns to binary using the median as the threshold
for col in numerical_columns:
    median_value = train_data[col].median()
    train_data[col] = train_data[col].apply(lambda x: 1 if x > median_value else 0)
    test_data[col] = test_data[col].apply(lambda x: 1 if x > median_value else 0)

# Adjust the heuristic functions to handle possible empty subsets
def entropy(data):
    """Calculate the entropy for a dataset, handling possible empty subsets."""
    if len(data) == 0:  # Handle empty subsets
        return 0
    label_col = data['y']
    counts = Counter(label_col)
    total = len(label_col)
    return -sum((count / total) * log2(count / total) for count in counts.values())

def majority_error(data):
    """Calculate the majority error for a dataset."""
    if len(data) == 0:  # Handle empty subsets
        return 0
    label_col = data['y']
    counts = Counter(label_col)
    majority_class_count = max(counts.values())
    total = len(label_col)
    return 1 - majority_class_count / total

def gini_index(data):
    """Calculate the Gini index for a dataset."""
    if len(data) == 0:  # Handle empty subsets
        return 0
    label_col = data['y']
    counts = Counter(label_col)
    total = len(label_col)
    return 1 - sum((count / total) ** 2 for count in counts.values())

# Modify the split_data function to ensure it returns valid DataFrames
def split_data(data, attribute, value):
    """Split the data based on the attribute and value."""
    return data[data[attribute] == value]

# ID3 algorithm implementation to handle both categorical and binary numerical attributes
class DecisionTreeID3Modified:
    def __init__(self, heuristic, max_depth=None):
        self.heuristic = heuristic
        self.max_depth = max_depth
        self.tree = None
    
    def _get_best_split(self, data, attributes):
        base_heuristic_value = self.heuristic(data)
        best_gain = 0
        best_attribute = None
        
        for attribute in attributes:
            values = data[attribute].unique()
            weighted_heuristic = 0
            for value in values:
                subset = split_data(data, attribute, value)
                weight = len(subset) / len(data)
                weighted_heuristic += weight * self.heuristic(subset)
            
            gain = base_heuristic_value - weighted_heuristic
            if gain > best_gain:
                best_gain = gain
                best_attribute = attribute
                
        return best_attribute

    def _build_tree(self, data, attributes, depth):
        label_counts = Counter(data['y'])
        if len(label_counts) == 1:
            return next(iter(label_counts))

        if len(attributes) == 0 or (self.max_depth is not None and depth == self.max_depth):
            return label_counts.most_common(1)[0][0]

        best_attribute = self._get_best_split(data, attributes)
        if best_attribute is None:
            return label_counts.most_common(1)[0][0]
        
        tree = {best_attribute: {}}
        remaining_attributes = [attr for attr in attributes if attr != best_attribute]
        for value in data[best_attribute].unique():
            subset = split_data(data, best_attribute, value)
            tree[best_attribute][value] = self._build_tree(subset, remaining_attributes, depth + 1)
        
        return tree

    def fit(self, data):
        attributes = data.columns[:-1]  # All columns except the label
        self.tree = self._build_tree(data, attributes, 0)

    def predict_single(self, example, tree):
        if not isinstance(tree, dict):
            return tree
        attribute = next(iter(tree))
        value = example[attribute]
        subtree = tree[attribute].get(value, None)
        if subtree is None:
            return None  # In case the value is not found in the training data
        return self.predict_single(example, subtree)

    def predict(self, data):
        return data.apply(lambda row: self.predict_single(row, self.tree), axis=1)

# Define heuristics
heuristics = {
    'information_gain': entropy,
    'majority_error': majority_error,
    'gini_index': gini_index
}

# Calculate errors for depths 1 to 16 and different heuristics
results = []

for heuristic_name, heuristic_function in heuristics.items():
    for depth in range(1, 17):
        model = DecisionTreeID3Modified(heuristic_function, depth)
        model.fit(train_data)
        
        train_predictions = model.predict(train_data)
        test_predictions = model.predict(test_data)
        
        train_error = np.mean(train_predictions != train_data['y'])
        test_error = np.mean(test_predictions != test_data['y'])
        
        results.append((heuristic_name, depth, train_error, test_error))

# Create a DataFrame to display the results for the tree
results_df = pd.DataFrame(results, columns=['Heuristic', 'Max Depth', 'Train Error', 'Test Error'])

# Display the results
print(results_df)


           Heuristic  Max Depth  Train Error  Test Error
0   information_gain          1       0.1192      0.1248
1   information_gain          2       0.1060      0.1114
2   information_gain          3       0.1006      0.1080
3   information_gain          4       0.0800      0.1264
4   information_gain          5       0.0624      0.1420
5   information_gain          6       0.0480      0.1508
6   information_gain          7       0.0372      0.1604
7   information_gain          8       0.0292      0.1658
8   information_gain          9       0.0222      0.1708
9   information_gain         10       0.0182      0.1752
10  information_gain         11       0.0154      0.1760
11  information_gain         12       0.0140      0.1788
12  information_gain         13       0.0136      0.1790
13  information_gain         14       0.0136      0.1790
14  information_gain         15       0.0136      0.1790
15  information_gain         16       0.0136      0.1790
16    majority_error          1